In [1]:
from ultralytics import YOLO

# --- 1. Cargar el modelo base yolo11n.pt ---
# Asumiendo que tienes este archivo y es compatible.
try:
    model = YOLO('yolo11n.pt') # Carga los pesos base de yolo11n
    print("Modelo base 'yolo11n.pt' cargado correctamente.")
except Exception as e:
    print(f"Error cargando el modelo base 'yolo11n.pt': {e}")
    print("Asegúrate de que el archivo existe y es compatible con Ultralytics.")
    exit()

# --- 2. Iniciar Entrenamiento ---
# Entrena el modelo usando tu archivo 'matriculas_solo.yaml'
# Este YAML DEBE tener nc: 1 y names: ['matricula']
results = model.train(
    data='matriculas.yaml',    # Ruta al archivo YAML (solo 1 clase: matricula)
    epochs=50,                    # Número de épocas (ajusta según necesites)
    imgsz=640,                    # Tamaño de imagen
    batch=4,                     # Tamaño del lote
    name='modelo_matriculas_yolo11', # Nombre para la carpeta de resultados
    exist_ok=True                 # Permite sobrescribir
)

print("\n¡Entrenamiento para SOLO matrículas (base yolo11n) completado!")
# La ruta real donde se guardan los pesos
print(f"Mejores pesos del modelo guardados en: runs/detect/{results.save_dir}/weights/best.pt")

Modelo base 'yolo11n.pt' cargado correctamente.
New https://pypi.org/project/ultralytics/8.3.222 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.215  Python-3.9.23 torch-2.8.0+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=matriculas.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mo

In [2]:
from ultralytics import YOLO
import cv2
import glob
import os
import matplotlib.pyplot as plt

# --- CONFIGURACIÓN DE MODELOS, dibujar imagen ---

# Carga el modelo general YOLOv8 pre-entrenado (para vehículos)
try:
    model_general_coches = YOLO('yolov8n.pt') # Usamos yolov8n estándar para coches
    print("Modelo general (yolov8n.pt) para coches cargado correctamente.")
except Exception as e:
    print(f"Error cargando modelo general 'yolov8n.pt': {e}")
    exit()

# Carga TU modelo entrenado (el que se basó en yolo11n.pt)
# ¡¡IMPORTANTE!! Ajusta esta ruta al 'best.pt' del entrenamiento anterior.
try:
    # Usa el nombre que le diste al entrenamiento ('modelo_matriculas_yolo11')
    ruta_modelo_matriculas_entrenado = 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' # <--- AJUSTA ESTA RUTA
    model_matriculas = YOLO(ruta_modelo_matriculas_entrenado)
    print(f"Modelo entrenado para matrículas '{ruta_modelo_matriculas_entrenado}' cargado correctamente.")
except Exception as e:
    print(f"Error cargando tu modelo entrenado de matrículas '{ruta_modelo_matriculas_entrenado}': {e}")
    print("Asegúrate de que la ruta apunta al archivo 'best.pt' correcto.")
    exit()

# Clases de interés del modelo general (vehículos COCO)
coco_vehicle_classes = [2, 3, 5, 7] # car, motorcycle, bus, truck

# Colores y confianzas
color_vehiculo = (255, 0, 0) # Azul
color_matricula = (0, 255, 0) # Verde
conf_vehiculo = 0.40
conf_matricula = 0.1

# --- DEFINIR IMÁGENES DE PRUEBA Y SALIDA ---
ruta_carpeta_imagenes_test = './TGC_RBNW/test/images/*'
rutas_imagenes_test = glob.glob(ruta_carpeta_imagenes_test)
if not rutas_imagenes_test:
    print(f"Error: No se encontraron imágenes en '{ruta_carpeta_imagenes_test}'.")
    exit()
output_dir_deteccion_combinada = 'resultados_deteccion_combinada_y11base'
os.makedirs(output_dir_deteccion_combinada, exist_ok=True)

print(f"Procesando {len(rutas_imagenes_test)} imágenes...")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

# --- PROCESAR IMÁGENES Y DIBUJAR ---
for img_path in rutas_imagenes_test:
    base_filename = os.path.basename(img_path)
    # print(f"Procesando: {base_filename}") # Puedes descomentar para ver el progreso

    frame = cv2.imread(img_path)
    if frame is None: continue
    frame_annotated = frame.copy()

    # --- 1. Detectar VEHÍCULOS ---
    results_coches = model_general_coches.predict(frame, classes=coco_vehicle_classes, conf=conf_vehiculo, verbose=False)
    if len(results_coches) > 0 and len(results_coches[0].boxes) > 0:
        for box_veh in results_coches[0].boxes:
            coords_veh = box_veh.xyxy.cpu().numpy().astype(int)[0]
            x1_v, y1_v, x2_v, y2_v = coords_veh
            conf_v = box_veh.conf.cpu().numpy()[0]
            class_id_v = int(box_veh.cls.cpu().numpy()[0])
            label_veh = model_general_coches.names[class_id_v]
            cv2.rectangle(frame_annotated, (x1_v, y1_v), (x2_v, y2_v), color_vehiculo, 2)
            cv2.putText(frame_annotated, f'{label_veh} {conf_v:.2f}', (x1_v, y1_v - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_vehiculo, 2)

    # --- 2. Detectar MATRÍCULAS ---
    results_matricula = model_matriculas.predict(frame, conf=conf_matricula, verbose=False)
    if len(results_matricula) > 0 and len(results_matricula[0].boxes) > 0:
        for box_mat in results_matricula[0].boxes:
            coords_mat = box_mat.xyxy.cpu().numpy().astype(int)[0]
            mx1, my1, mx2, my2 = coords_mat
            conf_m = box_mat.conf.cpu().numpy()[0]
            label_mat = model_matriculas.names[0] # Siempre será 'matricula'
            cv2.rectangle(frame_annotated, (mx1, my1), (mx2, my2), color_matricula, 2)
            cv2.putText(frame_annotated, f'{label_mat} {conf_m:.2f}', (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_matricula, 2)

    # --- GUARDAR ---
    output_path = os.path.join(output_dir_deteccion_combinada, f"comb_{base_filename}")
    cv2.imwrite(output_path, frame_annotated)

# --- FINALIZACIÓN ---
print("\nProceso de detección combinada completado.")
print(f"Resultados guardados en: {output_dir_deteccion_combinada}")

Modelo general (yolov8n.pt) para coches cargado correctamente.
Modelo entrenado para matrículas 'runs/detect/modelo_matriculas_yolo11/weights/best.pt' cargado correctamente.
Procesando 24 imágenes...
Resultados guardados en: resultados_deteccion_combinada_y11base

Proceso de detección combinada completado.
Resultados guardados en: resultados_deteccion_combinada_y11base
